In [2]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [3]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [11]:
info_dir = f'/search/odin/publicData/CloudS/mkyuwen/rank/infos/video/11/'
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
hour = 2020051812
info_path = f'{info_dir}/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
df['dur'] = df.duration
df['mark'] = 'tuwen'
df = df[df.product_data=='sgsapp']
df.head()

/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8/2020051812/valid.csv


,doc_topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,id,impression_time,...,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur,mark
0,0,7,1589682824,1,3127032093734048289,3057,20517a042fLFF7,48,4cdc866046040056414\t20517a042fLFF7,1589775989,...,1,1.0,0,sgsapp,0.020801,0.156259,0.124285,2020051812,48,tuwen
1,0,5,1588475101,12,-3999833754333635020,3019,20503b042fYyF0,0,776b862746039913145\t20503b042fYyF0,1589775584,...,0,1.0,0,sgsapp,0.007036,0.078404,0.095078,2020051812,0,tuwen
2,0,11,1587891755,3,-855708437921590511,2851,20426h043PkRXL,0,1d18864387023639335\t20426h043PkRXL,1589775237,...,0,1.0,0,sgsapp,0.009965,0.264141,0.016384,2020051812,0,tuwen
3,0,15,1589719647,1,5677494131456343107,8007,20517l0K43d7Oi,0,92d8864292031704610\t20517l0K43d7Oi,1589773384,...,0,1.0,0,sgsapp,0.021338,0.171486,0.108449,2020051812,0,tuwen
4,0,3,1585748488,2,-7517032370093146190,3057,2040200K4rkVFj,0,b63d1553499059947|ba74fc\t2040200K4rkVFj,1589772714,...,0,1.0,0,sgsapp,0.024058,0.071473,0.653928,2020051812,0,tuwen


In [12]:
df['click'] = (df.dur != 0).astype(int)

In [13]:
df[df.abtestid==5].video_time.mean()

304.82522052927027

In [14]:
df[df.abtestid==15].video_time.mean()

298.3294884843002

In [15]:
df[df.abtestid==8].video_time.mean()

278.89006191458964

In [5]:
# kv = KV()
# titles = kv.get_titles(list(set(df.docid.values)))
# df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

In [5]:
pd.set_option('display.max_rows', 1000)
pd.set_option("display.precision", 8)

In [6]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [7]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [8]:
mean_duration = df[df.dur > 0].dur.mean()

In [9]:
mean_duration

115.91473298277775

In [10]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [11]:
import math
df.pred = df.pred.apply(lambda x: math.sqrt(x))

In [12]:
cold_mids = set(df[df.cold==True].mid.values)

In [13]:
len(set(df.mid.values))

61898

In [14]:
len(cold_mids)

4300

In [17]:
len(set(df.impression.values)) 

158498

In [18]:
len(set(df.mid.values))

61898

In [19]:
len(set(df.impression.values))  / len(set(df.mid.values))

2.5606320074962032

In [20]:
len(set(df[df.dur!=0].mid.values))

45017

In [21]:
len(set(df[df.dur!=0].mid.values)) / len(set(df.mid.values))

0.7272771333484119

In [22]:
from projects.feed.rank.src.evaluate import evaluate_df as eval
import melt 

In [23]:
melt.init_flags()

In [25]:
df[df.abtestid.isin([4,5,6])].abtestid

3          4
5          4
7          4
33         6
34         6
          ..
1175771    6
1175975    6
1176077    5
1176128    4
1176638    5
Name: abtestid, Length: 77760, dtype: int64

In [27]:
result = eval(df[df.abtestid.isin([4,5,6])], group_by_impression=True, key='ori_lr_score', eval_click=False, eval_duration=False)
result 

{'auc': 0.6759705816886179,
 'mse': 0.30920069679892626,
 'mae': 0.39073022496121335,
 'time_auc': 0.6801654509475947,
 'mean': 0.6204140333990602,
 'loss/click': 7.722628021326354,
 'loss/dur': array(8.077366, dtype=float32),
 'stats/pos_ratio': 0.162988683127572,
 'vtime_auc': 0.4024540694521629,
 'finish_auc': 0.6545967644093093,
 'rcr_auc': 0.6841759506160702,
 'ptime_auc': 0.5017487058918892,
 'stats/num_instances': 77760,
 'stats/click_ratio': 0.162988683127572,
 'stats/time_per_show': 0.30661329324880726,
 'stats/time_per_click': 0.034237455118661886,
 'stats/time_per_user': 0.018270749602766617,
 'stats/vtime_per_show': 5.4026969914820375,
 'stats/vtime_per_click': 5.02265522374989,
 'stats/vtime_per_user': 2.680329937377325,
 'click/time_auc': 0.4353762728532967,
 'click/loss/dur': nan,
 'click/mse': 1.5402289476936484,
 'click/rmse': -0.24105960682541294,
 'click/mae': 0.9831916572417346,
 'click/mean': 1.0005900494947646,
 'group/auc': 0.5764018657275739,
 'group/auc2': 0.59

In [28]:
result = eval(df[df.abtestid.isin([4,5,6])], group_by_impression=True, key='ol_pred', eval_click=False, eval_duration=False)
result 

{'auc': 0.7405553899381669,
 'mse': 0.012531429881353055,
 'mae': 0.044211987995940095,
 'time_auc': 0.7284824697034724,
 'mean': 0.029516563206512622,
 'loss/click': 0.5689903610639381,
 'loss/dur': array(0.11565489, dtype=float32),
 'stats/pos_ratio': 0.162988683127572,
 'vtime_auc': 0.5593716928859827,
 'finish_auc': 0.6727454835204445,
 'rcr_auc': 0.7096782600489903,
 'ptime_auc': 0.47118549972188406,
 'stats/num_instances': 77760,
 'stats/click_ratio': 0.162988683127572,
 'stats/time_per_show': 0.30661329324880726,
 'stats/time_per_click': 0.034237455118661886,
 'stats/time_per_user': 0.018270749602766617,
 'stats/vtime_per_show': 5.4026969914820375,
 'stats/vtime_per_click': 5.02265522374989,
 'stats/vtime_per_user': 2.680329937377325,
 'click/time_auc': 0.6287950630140222,
 'click/loss/dur': array(0.62977076, dtype=float32),
 'click/mse': 0.07437913607192649,
 'click/rmse': 0.7272746141776925,
 'click/mae': 0.15634072696077533,
 'click/mean': 0.055075317537803224,
 'group/auc': 

In [29]:
result = eval(df[df.abtestid.isin([4,5,6])], group_by_impression=True, key='pred', eval_click=False, eval_duration=False)
result 

{'auc': 0.7395198643747304,
 'mse': 0.028727362179103783,
 'mae': 0.13175963431278134,
 'time_auc': 0.7280307702901472,
 'mean': 0.13916671411176068,
 'loss/click': 0.3917828265869535,
 'loss/dur': array(0.19048119, dtype=float32),
 'stats/pos_ratio': 0.162988683127572,
 'vtime_auc': 0.5610117079282995,
 'finish_auc': 0.6728003684973658,
 'rcr_auc': 0.7094564970040151,
 'ptime_auc': 0.4649301823303711,
 'stats/num_instances': 77760,
 'stats/click_ratio': 0.162988683127572,
 'stats/time_per_show': 0.30661329324880726,
 'stats/time_per_click': 0.034237455118661886,
 'stats/time_per_user': 0.018270749602766617,
 'stats/vtime_per_show': 5.4026969914820375,
 'stats/vtime_per_click': 5.02265522374989,
 'stats/vtime_per_user': 2.680329937377325,
 'click/time_auc': 0.6324791102148668,
 'click/loss/dur': array(0.47738594, dtype=float32),
 'click/mse': 0.05402847571356041,
 'click/rmse': 0.7675597373225533,
 'click/mae': 0.16768400438108547,
 'click/mean': 0.20957207765968022,
 'group/auc': 0.53

In [35]:
df[df.abtestid.isin([4,5,6])].pred_dur.mean()

0.2146041075640701

In [34]:
df[df.abtestid.isin([4,5,6])].ol_pred_dur.mean()

0.2215394984728565

In [36]:
result = eval(df[df.abtestid.isin([15])], group_by_impression=True, key='ori_lr_score', eval_click=False, eval_duration=False)
result 

{'auc': 0.6778005798630091,
 'mse': 0.32603565268233214,
 'mae': 0.4097042005569797,
 'time_auc': 0.6798372172353317,
 'mean': 0.6491678467343435,
 'loss/click': 8.128125390254384,
 'loss/dur': array(8.506653, dtype=float32),
 'stats/pos_ratio': 0.1627491252091891,
 'vtime_auc': 0.3807620438580467,
 'finish_auc': 0.6563608372423932,
 'rcr_auc': 0.6873908746543967,
 'ptime_auc': 0.49277714905201453,
 'stats/num_instances': 26292,
 'stats/click_ratio': 0.1627491252091891,
 'stats/time_per_show': 0.30414378174879453,
 'stats/time_per_click': 0.03412016546616911,
 'stats/time_per_user': 0.017982375899499938,
 'stats/vtime_per_show': 5.228958123823522,
 'stats/vtime_per_click': 4.870059880239522,
 'stats/vtime_per_user': 2.5666712403951704,
 'click/time_auc': 0.40483821501200956,
 'click/loss/dur': nan,
 'click/mse': 1.6258776646876887,
 'click/rmse': -0.27509908034148034,
 'click/mae': 1.0255459695561178,
 'click/mean': 1.034015121945373,
 'group/auc': 0.5772415415021551,
 'group/auc2': 0.

In [37]:
result = eval(df[df.abtestid.isin([15])], group_by_impression=True, key='ol_pred', eval_click=False, eval_duration=False)
result 

{'auc': 0.7328393246817005,
 'mse': 0.012483271915383941,
 'mae': 0.04528043387453696,
 'time_auc': 0.7229115273455051,
 'mean': 0.030758132531286675,
 'loss/click': 0.5621785287844481,
 'loss/dur': array(0.11549122, dtype=float32),
 'stats/pos_ratio': 0.1627491252091891,
 'vtime_auc': 0.5704958646292028,
 'finish_auc': 0.6671196513593962,
 'rcr_auc': 0.7044567927493721,
 'ptime_auc': 0.46608461117308797,
 'stats/num_instances': 26292,
 'stats/click_ratio': 0.1627491252091891,
 'stats/time_per_show': 0.30414378174879453,
 'stats/time_per_click': 0.03412016546616911,
 'stats/time_per_user': 0.017982375899499938,
 'stats/vtime_per_show': 5.228958123823522,
 'stats/vtime_per_click': 4.870059880239522,
 'stats/vtime_per_user': 2.5666712403951704,
 'click/time_auc': 0.6335188220420018,
 'click/loss/dur': array(0.62324387, dtype=float32),
 'click/mse': 0.07378153136754141,
 'click/rmse': 0.7283724399705703,
 'click/mae': 0.1562414614382612,
 'click/mean': 0.05565224036098903,
 'group/auc': 0